In [1]:
# Import libraries
from datetime import datetime, timedelta
import numpy as np

In [2]:
# For the global variables
FILE = 'working_hours.txt'

DAILY_HOURS_TO_COVER = 8
TOTAL_HOURS_TO_COVER = 40

MAX_MINUTES_RECOMMENDED = 60 * DAILY_HOURS_TO_COVER
MAX_MINUTES_WEEKLY = 60 * TOTAL_HOURS_TO_COVER
MAX_MINUTES_OVERTIME = 570

In [3]:
# Get the current time
def get_current_time():
    return str(datetime.now().hour) + ':' + str(datetime.now().minute)

# Return the total time covered in Xh and Ym
def get_hours_minutes(total_minutes):

    hours = int(total_minutes/60)
    minutes = int(total_minutes % 60)

    return str(hours) + 'h ' + str(minutes) + 'm'

# Find the difference of two timeframes
def get_duration(times):
    
    minutes_covered = 0

    # Each time gap has two outcomes: 
    # 1. 1400-1800 (240 minutes) 
    # 2. 0800- (Current time)
    for hours in times:
        hours = hours.rstrip('\n')
        
        # Split the string based on '-'
        (start_time, end_time) = hours.split('-')
        FMT = '%H:%M'

        end_time = end_time if end_time != '' else get_current_time()

        # Get the time difference
        diff = datetime.strptime(end_time, FMT) - datetime.strptime(start_time, FMT)
        minutes_covered += diff.seconds/60

    return int(minutes_covered)

def calculate_hours():
    total_covered = 0

    days_array = []
    minutes_array = []
    finishing_time_today = ''
    
    day_number = datetime.today().weekday()

    # Load text file
    with open(FILE) as file:

        # One line = Day, Time gap #1, Time gap #2, ..., Time gap n
        for index, line in enumerate(file):
            
            line_array = line.replace('\x00','').rstrip().split(',')
            
            day = line_array[0]
            day_coverage = line_array[1:]

            minutes_covered = get_duration(day_coverage)
            
            days_array.append({
                'day': day,
                'minutes_covered': minutes_covered,
                'hours_covered': get_hours_minutes(minutes_covered),
                'coverage': day_coverage
            })
            
            if index == day_number:
                
                remaining = MAX_MINUTES_RECOMMENDED - minutes_covered
                remaining = 0 if remaining <= 0 else remaining

                finishing_time_today = datetime.now() + timedelta(minutes = remaining)

            total_covered += minutes_covered

    # Calculate overall week stats
    total_covered = total_covered if total_covered <= MAX_MINUTES_WEEKLY else MAX_MINUTES_WEEKLY

    return (total_covered, days_array, finishing_time_today)

In [4]:
# Find:
# - the total hours covered
# - the days covered so far and the breakdown of minutes per day

(total_covered, days_array, finishing_time_today) = calculate_hours()

In [5]:
# Convert the days_array to dataframe
import pandas as pd

days_df = pd.DataFrame(days_array)
print(days_df)
print('\n\n')
print('FINISHING TIME TODAY: ', finishing_time_today.time().strftime("%H:%M:%S"))

   day  minutes_covered hours_covered                    coverage
0  Mon              487         8h 7m               [09:20-17:27]
1  Tue              430        7h 10m  [09:18-11:52, 12:07-16:43]
2  Wed              516        8h 36m  [08:22-11:57, 12:11-17:12]
3  Thu              493        8h 13m  [09:02-12:06, 12:20-17:29]
4  Fri              496        8h 16m                [9:39-17:55]



FINISHING TIME TODAY:  21:36:33


In [34]:
# Bar chart
import plotly.express as px

fig = px.bar(days_df, 
             x='minutes_covered', y='day',
            hover_data=['coverage', 'hours_covered'],
            labels={
                'day': 'Day', 
                'minutes_covered': 'Total (Minutes)', 
                'hours_covered': 'Total (Hours)',
                'coverage': 'Day coverage'
            },
            orientation='h',
            height = 400)

fig.update_layout(
    title_text='Weekly hours calculation (Individual Days)',
    yaxis=dict(autorange="reversed"))

fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=MAX_MINUTES_RECOMMENDED,
            x1=MAX_MINUTES_RECOMMENDED,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="Green",
                width=3
            )
))

fig.update_shapes(dict(xref='x', yref='y'))

fig.show()

In [8]:
remaining_time = MAX_MINUTES_WEEKLY - total_covered if MAX_MINUTES_WEEKLY > total_covered else 0

# Convert to dataframe
total_calculation_df = pd.DataFrame([
    {
        'category': 'covered', 
        'amount': total_covered, 
        'amount_hrs': get_hours_minutes(total_covered)
    },
    {
        'category': 'remaining', 
        'amount': remaining_time,
        'amount_hrs': get_hours_minutes(remaining_time)
    }
])

print(total_calculation_df)

    category  amount amount_hrs
0    covered    2400     40h 0m
1  remaining       0      0h 0m


In [9]:
# Pie chart
import plotly.graph_objects as go

fig = go.Figure(go.Pie(
    name="",
    values = total_calculation_df['amount'],
    labels = total_calculation_df['category'],
    customdata = total_calculation_df['amount_hrs'],
    hovertemplate = "Category: %{label} <br>Total(minutes): %{value} </br>Total(hours): %{customdata}"

))

fig.update_layout(title_text='Overall weekly hours calculation: remaining vs covered')

fig.show()